In [131]:
from sqlalchemy import create_engine
import os
import sys
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get credentials
DBNAME = os.getenv('DBNAME')
DBUSER = os.getenv('DBUSER')
DBPASSWORD = os.getenv('DBPASSWORD')
DBHOST = os.getenv('DBHOST')
DBPORT = os.getenv('DBPORT')

# Create the database URI and engine
DATABASE_URI = f'postgresql+psycopg2://{DBUSER}:{DBPASSWORD}@{DBHOST}:{DBPORT}/{DBNAME}'
engine = create_engine(DATABASE_URI)

# Test the connection
try:
    with engine.connect() as connection:
        print("Connection to the database was successful!")
except Exception as e:
    print(f"Connection failed: {e}")

Connection to the database was successful!


In [143]:
import pandas as pd

# Define a function to load data from a specific table
def load_table(table_name):
    query = f"SELECT * FROM {table_name};"
    return pd.read_sql(query, engine)

# Load data from `xdr_data` table

df_xdr = load_table('xdr_data')

# Display the first few rows 

print(df_xdr.head())


      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          20247395.0          19111729.0   

In [149]:
# Count of each handset and get the top 10
top_handsets = df_xdr['Handset Type'].value_counts().head(10)
print("\nTop 10 Handsets:")
print(top_handsets)



Top 10 Handsets:
Handset Type
Huawei B528S-23A                59256
Apple iPhone 6S (A1688)         28257
Apple iPhone 6 (A1586)          27069
undefined                       26961
Apple iPhone 7 (A1778)          18978
Apple iPhone Se (A1723)         15561
Apple iPhone 8 (A1905)          14979
Apple iPhone Xr (A2105)         13704
Samsung Galaxy S8 (Sm-G950F)    13560
Apple iPhone X (A1901)          11439
Name: count, dtype: int64


In [150]:
# Count of each manufacturer and get the top 3
top_manufacturers = df_xdr['Handset Manufacturer'].value_counts().head(3)
print("\nTop 3 Manufacturers:")
print(top_manufacturers)



Top 3 Manufacturers:
Handset Manufacturer
Apple      178695
Samsung    122517
Huawei     103269
Name: count, dtype: int64


In [152]:
# Filter handsets for the top 3 manufacturers
top_manufacturers_list = top_manufacturers.index
filtered_handsets = df_xdr[df_xdr['Handset Manufacturer'].isin(top_manufacturers_list)]

# Get top 5 handsets per manufacturer
top_handsets_per_manufacturer = {}
for manufacturer in top_manufacturers_list:
    top_handsets_for_manufacturer = filtered_handsets[filtered_handsets['Handset Manufacturer'] == manufacturer]
    top_5_handsets = top_handsets_for_manufacturer['Handset Type'].value_counts().head(5)
    top_handsets_per_manufacturer[manufacturer] = top_5_handsets

print("\nTop 5 Handsets for Each Manufacturer:")
for manufacturer, handsets in top_handsets_per_manufacturer.items():
    print(f"\n{manufacturer}:")
    print(handsets)



Top 5 Handsets for Each Manufacturer:

Apple:
Handset Type
Apple iPhone 6S (A1688)    28257
Apple iPhone 6 (A1586)     27069
Apple iPhone 7 (A1778)     18978
Apple iPhone Se (A1723)    15561
Apple iPhone 8 (A1905)     14979
Name: count, dtype: int64

Samsung:
Handset Type
Samsung Galaxy S8 (Sm-G950F)    13560
Samsung Galaxy A5 Sm-A520F      11172
Samsung Galaxy J5 (Sm-J530)     11088
Samsung Galaxy J3 (Sm-J330)     10452
Samsung Galaxy S7 (Sm-G930X)     9597
Name: count, dtype: int64

Huawei:
Handset Type
Huawei B528S-23A                  59256
Huawei E5180                       6237
Huawei P20 Lite Huawei Nova 3E     6063
Huawei P20                         4440
Huawei Y6 2018                     2991
Name: count, dtype: int64


In [162]:
import pandas as pd

# Ensure all necessary columns are numeric before aggregation
numeric_columns = [
    "Dur. (ms)", "Total DL (Bytes)", "Total UL (Bytes)",
    "Social Media DL (Bytes)", "Social Media UL (Bytes)",
    "Google DL (Bytes)", "Google UL (Bytes)",
    "Email DL (Bytes)", "Email UL (Bytes)",
    "Youtube DL (Bytes)", "Youtube UL (Bytes)",
    "Netflix DL (Bytes)", "Netflix UL (Bytes)",
    "Gaming DL (Bytes)", "Gaming UL (Bytes)",
    "Other DL (Bytes)", "Other UL (Bytes)"
]

# Convert these columns to numeric
for col in numeric_columns:
    df_xdr[col] = pd.to_numeric(df_xdr[col], errors='coerce')

# Aggregate user behavior data
user_agg = df_xdr.groupby('IMEI').agg({
    "Bearer Id": "count",  # Number of xDR sessions
    "Dur. (ms)": "sum",  # Total session duration in ms
    "Total DL (Bytes)": "sum",  # Total download data in Bytes
    "Total UL (Bytes)": "sum",  # Total upload data in Bytes
    "Social Media DL (Bytes)": "sum",
    "Social Media UL (Bytes)": "sum",
    "Google DL (Bytes)": "sum",
    "Google UL (Bytes)": "sum",
    "Email DL (Bytes)": "sum",
    "Email UL (Bytes)": "sum",
    "Youtube DL (Bytes)": "sum",
    "Youtube UL (Bytes)": "sum",
    "Netflix DL (Bytes)": "sum",
    "Netflix UL (Bytes)": "sum",
    "Gaming DL (Bytes)": "sum",
    "Gaming UL (Bytes)": "sum",
    "Other DL (Bytes)": "sum",
    "Other UL (Bytes)": "sum"
}).reset_index()

print("\nAggregated User Data:")
print(user_agg.head())

# Handle missing values
print("\nMissing Values in Aggregated Data:")
print(user_agg.isnull().sum())

# Fill missing values with column means
user_agg.fillna(user_agg.mean(), inplace=True)

print("\nAggregated User Data After Handling Missing Values:")
print(user_agg.head())



Aggregated User Data:
           IMEI  Bearer Id  Dur. (ms)  Total DL (Bytes)  Total UL (Bytes)  \
0  4.400152e+11          3   223197.0      5.891552e+08       118275342.0   
1  4.402138e+11          3   259197.0      4.009838e+08        74568963.0   
2  1.194901e+12          3   263397.0      2.576678e+09       133228674.0   
3  1.198201e+12          3   169032.0      2.373087e+09       164523141.0   
4  1.202601e+12          3    54012.0      2.387488e+09       114211839.0   

   Social Media DL (Bytes)  Social Media UL (Bytes)  Google DL (Bytes)  \
0                7591818.0                  71904.0         27376707.0   
1                3903111.0                  31173.0         25561266.0   
2               10322106.0                  64746.0         25230423.0   
3                6465930.0                 141078.0         15473103.0   
4                1069329.0                 122187.0         17687562.0   

   Google UL (Bytes)  Email DL (Bytes)  Email UL (Bytes)  Youtube DL 